In [1]:
!git clone https://github.com/delmedigo88/SpaceGen.git

Cloning into 'SpaceGen'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 24 (delta 6), reused 23 (delta 5), pack-reused 0
Receiving objects: 100% (24/24), 16.61 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
import re
def replace_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text)

def remove_ellipsis_after_space(text):
    return re.sub(r' (?=\.{3})', '', text)

def remove_space_before_dot(text):
    return re.sub(r' (?=\.)', '', text)

In [11]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalFocalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from SpaceGen.SpaceGen_preprocessing import SpaceGen_preprocessing as SgPrep
from SpaceGen.SpaceGen_OneHotVectorizer import SpaceGen_OneHotVectorizer as SgOHV
from SpaceGen.SpaceGenerator import SpaceGenerator as SpaceGen
import warnings

warnings.filterwarnings('ignore')


def preprocess_text(raw_text, sg_prep):
    correct_txt = re.sub(r'[^a-zA-Z.,:\'?;!]', ' ', raw_text.strip())
    correct_txt = replace_multiple_spaces(correct_txt)
    correct_txt = remove_ellipsis_after_space(correct_txt)
    correct_txt = remove_space_before_dot(correct_txt)

    wrong_txt = re.sub(r'[^a-zA-Z.,:\'?;!]', '', correct_txt).replace(' ','')

    W = sg_prep.to_bytes_list(wrong_txt)
    C = sg_prep.to_bytes_list(correct_txt)
    D = sg_prep.create_decision_vector(W, C)

    letters = sg_prep.to_one_hot_df(W, D).to_numpy()[:,0]
    past = sg_prep.sliding_window_past(letters, window_size= sg_prep.past_capacity)
    future = sg_prep.sliding_window_future(letters, window_size=sg_prep.future_capacity)
    letters = letters.reshape(-1, 1)

    X = np.concatenate((past, letters, future), axis= 1)
    y = sg_prep.to_one_hot_df(W, D).to_numpy()[:,1:3]

    one_hot_X = SgOHV.X_to_one_hot_matrix(X, vocabulary=sg_prep.vocabulary, max_len=sg_prep.num_features)
    X = np.stack(one_hot_X)

    return X, y

def build_model(input_shape, vocabulary_size):
    model = Sequential([
        Bidirectional(LSTM(units=100, activation='tanh', return_sequences=True), input_shape=input_shape),
        Dropout(0.2),
        Bidirectional(LSTM(units=50, activation='tanh')),
        Dropout(0.2),
        Dense(units=2, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=0.003)
    model.compile(optimizer=optimizer, loss=CategoricalFocalCrossentropy(), metrics=['accuracy'])
    return model

def train_incremental_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=320, validation_split=0.2, verbose=True)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test Accuracy: {accuracy}')
    return model, loss, accuracy

def generate_classification_report(model, X_test, y_test):
    y_predict = model.predict(X_test)
    y_p = np.argmax(y_predict, axis=1)
    y_t = np.argmax(y_test, axis=1)
    print("\nClassification report:\n")
    print(classification_report(y_t, y_p))
    print(f"\nROC AUC score: {roc_auc_score(y_t, y_p):.1%}")


def quick_check_model(model, sg_prep):
    space_generator = SpaceGen(model=model, vocabulary=sg_prep.vocabulary)
    text = '''It 'sno t ab out w haty ouw antto do,it ' sab outwh at yo u wan ttobe...'''
    print(space_generator.fix_text(text))
    text_3 = '''Curiosityispartofhumannature.Oneofthefirstquestionschildrenlearntoaskis
                "why?"Asadults,wecontinuetowonder.Usingempiricalmethods,psychologistsapplythatunive
                rsalcuriositytocollectandinterpretresearchdatatobetterunderstandandsolvesomeofsocie
                ty’smostchallengingproblems.It’sdifficult,ifnotimpossible,tothinkofafacetoflifewher
                epsychologyisnotinvolved.Psychologistsemploythescientificmethod—statingthequestion,
                offeringatheoryandthenconstructingrigorouslaboratoryorfieldexperimentstotestthehypo
                thesis.Psychologistsapplytheunderstandinggleanedthroughresearchtocreateevidence-bas
                edstrategiesthatsolveproblemsandimprovelives.Theresultisthatpsychologicalscienceunv
                eilsnewandbetterwaysforpeopletoexistandthriveinacomplexworld.'''
    print(space_generator.fix_text(text_3))

In [12]:
past_capacity = 5
future_capacity = 5
chunk_size = 1000 * 1000
text_size = chunk_size * 5

with open('/content/SpaceGen/Resources/Raw/MovieCorpus.txt', 'r') as txt_file:
    content = txt_file.read(text_size)

sg_prep = SgPrep(content=content, size=text_size, past_capacity=past_capacity, future_capacity=future_capacity)
sg_prep.create_vocabulary(content)

model = build_model(input_shape=(1, sg_prep.num_features * len(sg_prep.vocabulary)),
                    vocabulary_size=len(sg_prep.vocabulary))

for i in range(0, len(content), chunk_size):
    chunk = content[i:i+chunk_size]
    X, y = preprocess_text(chunk, sg_prep)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=42)

    X_train = X_train.reshape((X_train.shape[0], 1, sg_prep.num_features * len(sg_prep.vocabulary)))
    X_test = X_test.reshape((X_test.shape[0], 1, sg_prep.num_features * len(sg_prep.vocabulary)))

    model.fit(X_train, y_train, epochs=10, batch_size=320, validation_split=0.2, verbose=True)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test Accuracy: {accuracy}')
    generate_classification_report(model, X_test, y_test)
    quick_check_model(model, sg_prep)

Epoch 1/10
1714/1714 [==============================] - 46s 24ms/step - loss: 0.0089 - accuracy: 0.9470 - val_loss: 0.0054 - val_accuracy: 0.9710
Epoch 2/10
1714/1714 [==============================] - 39s 23ms/step - loss: 0.0045 - accuracy: 0.9756 - val_loss: 0.0042 - val_accuracy: 0.9782
Epoch 3/10
1714/1714 [==============================] - 39s 23ms/step - loss: 0.0032 - accuracy: 0.9823 - val_loss: 0.0039 - val_accuracy: 0.9808
Epoch 4/10
1714/1714 [==============================] - 40s 23ms/step - loss: 0.0025 - accuracy: 0.9862 - val_loss: 0.0040 - val_accuracy: 0.9816
Epoch 5/10
1714/1714 [==============================] - 41s 24ms/step - loss: 0.0020 - accuracy: 0.9889 - val_loss: 0.0045 - val_accuracy: 0.9815
Epoch 6/10
1714/1714 [==============================] - 40s 23ms/step - loss: 0.0017 - accuracy: 0.9906 - val_loss: 0.0041 - val_accuracy: 0.9823
Epoch 7/10
1714/1714 [==============================] - 41s 24ms/step - loss: 0.0014 - accuracy: 0.9919 - val_loss: 0.0047 -

In [13]:
#save model with pickle
import pickle
with open('SpaceGen_1.5.pkl', 'wb') as f:
    pickle.dump(model, f)

# load model
with open('SpaceGen_1.5.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [16]:
#save vocabulary with  pickle
import pickle
with open('SpaceGen_1.5_vocabulary.pkl', 'wb') as f:
    pickle.dump(sg_prep.vocabulary, f)

# load vocabulary
with open('SpaceGen_1.5_vocabulary.pkl', 'rb') as f:
    loaded_vocabulary = pickle.load(f)

In [17]:
space_gen = SpaceGen(model=loaded_model, vocabulary=loaded_vocabulary)

In [37]:
%%timeit
space_gen.fix_text('''Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea

 ''')

7.72 s ± 369 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
text = '''Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 Pl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthisseaPl eas edon'tca ll mea ny mor eIdon'twanta nycontact withyou. so... plea se.butan y ways,I wantedtoa skoneonelastquestionifImay...What'sthepurposeoflife?Ifnotlookingatthisframe-lockedwindows?youshouldknowtheanswerbutyouareafraidofmeforsomereasonthatIdon'tknow.Comeon!Don'tbescaredhoneyI'mnotgonnabiteyouYourey esareblueasthissea
 '''
print(f"{len(text) // 7.72} letters in sec")

4593.0 letters in sec
